This notebook illustrates how to respond to mouse events using an SVG canvas widget.

The goal of this exercise is to create an SVG canvas where the user can add circles by clicking,
move circles by using control-mousedown, and change the radius of a circle by using shift-mousedown.

In [1]:
# make it runable without an install.
import sys
if ".." not in sys.path:
    sys.path.append("..")

# Remember to install the required javascript.
from jp_svg_canvas import canvas
canvas.load_javascript_support()

<IPython.core.display.Javascript object>

In [2]:
import pprint
from random import randint
import ipywidgets as widgets
from IPython.display import display
import sys
import math

class Circles(object):
    """
    Display an SVG canvas which responds to mouse events
    as described above and also an informational textbox
    showing mouse event information dictionaries passed from
    javascript.
    """
    
    def __init__(self):
        "Construct the widgets to display and connect the event callback."
        self.selected_circle_name = None
        self.name_to_center = {}
        self.min_radius = 10
        self.count = 0
        self.info_area = widgets.Textarea()
        self.info_area.value = "Please click the canvas."
        # Create a canvas.
        self.svg = canvas.SVGCanvasWidget()
        # Respond to these mouse events.
        self.svg.watch_event = "click mousedown mouseup mousemove mouseover"
        # Connect a default callback to respond to all events.
        self.svg.default_event_callback = self.event_callback
        self.svg.add_style("background-color", "black")
        self.assembly = widgets.VBox(children=[self.info_area, self.svg])
        
    def display(self):
        display(self.assembly)
        
    def event_callback(self, info):
        "Respond to a mouse event over the canvas."
        # Get the name of the object most closely related to the event.
        name = info.get("name", "")
        selected = self.selected_circle_name
        # Get flags for whether shift or control keys are pressed.
        shift = info.get("shiftKey")
        control = info.get("ctrlKey")
        # get the position of the event in the SVG coordinate space
        x, y = info["svgX"], info["svgY"]
        # Get the event type.
        typ = info.get("type")
        # Show the event information dictionary in the info text box.
        self.info_area.value = repr((typ, name, selected, shift, control, x,y)) + "\n" + pprint.pformat(info)
        if not (shift or control or selected) and typ == "click":
            # Create a circle
            # with a random color not too dark.
            color = "#%02x%02x%02x" % (randint(100,255), randint(100,255), randint(100,255))
            # with the next name
            new_name = "circle_" + str(self.count)
            self.count += 1
            self.svg.circle(new_name, x, y, self.min_radius, color)
            self.name_to_center[new_name] = (x, y)
            # Remember to flush the command buffer!
            self.svg.send_commands()
            self.selected_circle_name = None
        elif typ == "mousedown" and name.startswith("circle") and (shift or control):
            self.selected_circle_name = name
        elif typ == "mouseup" or not (shift or control):
            # unselect any selected circle.
            self.selected_circle_name = None
        elif selected and shift and typ == "mousemove":
            # Change the radius of the selected circle.
            (cx, cy) = self.name_to_center[selected]
            dist = math.sqrt((x - cx) ** 2 + (y - cy) ** 2)
            new_radius = max(self.min_radius, dist)
            # change the "r" attribute of the SVG circle element associated with the name.
            atts = {"r": new_radius}
            self.svg.change_element(selected, atts)
            # remember to flush the command buffer!
            self.svg.send_commands()
        elif (name or selected) and control and typ == "mousemove":
            # Change the center of the targeted circle.
            target = (name or selected)
            # select the circle in case the mouse moves too fast.
            self.selected_circle_name = target
            atts = {"cx": x, "cy": y}
            self.svg.change_element(target, atts)
            self.name_to_center[target] = (x, y)
            self.svg.send_commands()


In [3]:
example = Circles()
example.display()

Click on the canvas to add a circle.

Hold down the Control key over a circle and move the mouse to change a circle location.

Hold down the Shift key, mouse down and move the mouse to change the sizeof a circle.